In [4]:
from scMethtools import preprocessing as pp

TypeError: unsupported operand type(s) for |: 'type' and 'NoneType'

In [16]:
pp.import_bed_file(
    data_dir ='D:\\Test/GSE56789/raw/',
    output_dir = 'D:\\Test/GSE56789/temp/',
)
pp.convert_coo_file_to_matrix('D:\\Test/GSE56789/temp','chr1')

AttributeError: module 'scMethtools.preprocessing' has no attribute 'import_bed_file'

In [3]:
!python --version

Python 3.9.17


In [1]:
import numpy as np
from sklearn.linear_model import ridge_regression

def inv_logit_mat(x, min=0.0, max=1.0,):
    # the inverse logit transformation function
    p = np.exp(x) / (1.0+np.exp(x))
    which_large = np.isnan(p) & (~np.isnan(x))
    p[which_large] = 1.0
    return p*(max-min)+min

def sparse_logistic_pca(
        dat, lbd=0.0006, k=2, verbose=False, max_iters=100, crit=1e-5,
        randstart=False, procrustes=True, lasso=True,
):
    """
    A Python implementation of the sparse logistic PCA of the following paper:
        Lee, S., Huang, J. Z., & Hu, J. (2010). Sparse logistic principal components analysis for binary data.
        The annals of applied statistics, 4(3), 1579.

    This implementation is migrated from this R package:
        https://github.com/andland/SparseLogisticPCA

    Args:
        dat: input data, n*d numpy array where n is the numbers of samples and d is the feature dimensionality
        lbd: the lambda value, higher value will lead to more sparse components
        k: the dimension after reduction
        verbose: print log or not
        max_iters: maximum number of iterations
        crit: the minimum difference criteria for stopping training
        randstart: randomly initialize A, B, mu or not
        procrustes: procrustes
        lasso: whether to use LASSO solver

    Returns: a dict containing the results

    """

    ### Initialize q
    q = 2*dat-1
    q[np.isnan(q)] = 0.0
    n,d = dat.shape

    ### Initialize mu, A, B
    if not randstart:
        mu = np.mean(q,axis=0)
        udv_u, udv_d, udv_v = np.linalg.svd(q-np.mean(q,axis=0), full_matrices=False)
        A = udv_u[:,0:k].copy()
        B = np.matmul(udv_v[0:k,:].T, np.diag(udv_d[0:k]))
    else:
        mu = np.random.normal(size=(d,))
        A = np.random.uniform(low=-1.0, high=1.0, size=(n,k,))
        B = np.random.uniform(low=-1.0, high=1.0, size=(d,k,))

    loss_trace = dict()

    ## loop to optimize the loss, see Alogrithm 1 in the paper
    for m in range(max_iters):

        last_mu, last_A, last_B = mu.copy(), A.copy(), B.copy()

        theta = np.outer(np.ones(n), mu) + np.matmul(A, B.T)
        X = theta+4*q*(1-inv_logit_mat(q*theta))
        Xcross = X - np.matmul(A, B.T)
        mu = np.matmul((1.0/n) * Xcross.T, np.ones(n))

        theta = np.outer(np.ones(n), mu) + np.matmul(A, B.T)
        X = theta+4*q*(1-inv_logit_mat(q*theta))
        Xstar = X-np.outer(np.ones(n), mu)

        if procrustes:
            M_u, M_d, M_v = np.linalg.svd(np.matmul(Xstar, B), full_matrices=False)
            A = np.matmul(M_u, M_v)
        else:
            A = Xstar @ B @ np.linalg.inv(B.T @ B)
            A, _ = np.linalg.qr(A)

        theta = np.outer(np.ones(n), mu) + A @ B.T
        X = theta + 4 * q * (1 - inv_logit_mat(q * theta))
        Xstar = X-np.outer(np.ones(n), mu)

        if lasso:
            B_lse = Xstar.T @ A
            B = np.sign(B_lse) * np.maximum(0.0, np.abs(B_lse)-4*n*lbd)
        else:
            C = Xstar.T @ A
            B = (np.abs(B) / (np.abs(B)+4*n*lbd)) * C

        q_dot_theta = q*(np.outer(np.ones(n),mu) + A @ B.T)
        loglike = np.sum(np.log(inv_logit_mat(q_dot_theta))[~np.isnan(dat)])
        penalty = n*lbd*np.sum(abs(B))
        loss_trace[str(m)] = (-loglike+penalty) / np.sum(~np.isnan(dat))

        if verbose:
            print(f"Iter: {m} - Loss: {loss_trace[str(m)]:.4f}, NegLogLike: {-loglike:.4f}, Penalty: {penalty:.4f} ")

        if m>3:
            if loss_trace[str(m-1)] - loss_trace[str(m)] < crit:
                break

    if loss_trace[str(m-1)] < loss_trace[str(m)]:
        mu, A, B, m = last_mu, last_A, last_B, m-1

        q_dot_theta = q*(np.outer(np.ones(n),mu) + A @ B.T)
        loglike = np.sum(np.log(inv_logit_mat(q_dot_theta))[~np.isnan(dat)])

    zeros = sum(np.abs(B))
    BIC = -2.0*loglike+np.log(n)*(d+n*k+np.sum(np.abs(B)>=1e-10))

    res = {
        "mu":mu, "A":A, "B":B, "zeros":zeros,
        "BIC":BIC, "iters":m, "loss_trace":loss_trace, "lambda":lbd,
    }

    return res

class SparseLogisticPCA(object):
    """
    A warper class of sparse logistic PCA, which provides the fit, transform and fit_transform methods
    """
    def __init__(
            self, lbd=0.0001, n_components=2, verbose=False, max_iters=100, crit=1e-5,
            randstart=False, procrustes=True, lasso=True,
            ridge_alpha=0.01,
    ):
        """
        Args:
            lbd: the lambda value, higher value will lead to more sparse components
            n_components: the dimension after reduction, i.e. k in the origin paper
            verbose: print log or not
            max_iters: maximum number of iterations
            crit: the minimum difference criteria for stopping training
            randstart: randomly initialize A, B, mu or not
            procrustes: procrustes
            lasso: whether to use LASSO solver
            ridge_alpha: Amount of ridge shrinkage to apply in order to improve conditioning when
                calling the transform method.
        """
        self.lbd = lbd
        self.n_components = n_components
        self.verbose=verbose
        self.max_iters = max_iters
        self.crit = crit
        self.randstart = randstart
        self.procrustes = procrustes
        self.lasso=lasso
        self.ridge_alpha = ridge_alpha

    def fit(self, dat, verbose=False):
        """

        Args:
            dat: ndarray of shape (n_samples, n_features), the data to be fitted
            verbose: print log or not

        Returns:
            self

        """
        res = sparse_logistic_pca(
            dat, lbd=self.lbd, k=self.n_components, verbose=verbose,
            max_iters=self.max_iters, crit=self.crit,
            randstart=self.randstart, procrustes=self.procrustes,
            lasso=self.lasso,)

        self.mu, self.components_ = res['mu'], res['B'].T
        _, self.d = dat.shape

        components_norm = np.linalg.norm(self.components_, axis=1)[:, np.newaxis]
        components_norm[components_norm == 0] = 1
        self.components_ /= components_norm

        return self

    def transform(self, X):
        """

        Similar to Sparse PCA, the orthogonality of the learned components is not enforced in Sparse Logistic PCA,
            and hence one cannot use a simple linear projection.

        The origin paper does not describe how to transform the new data, and this implementation of transform
            function generally follows that of sklearn.decomposition.SparsePCA:
            https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparsePCA.html

        The handling of missing data (N/A) is not supported in this transform implementation.

        Args:
            X: ndarray of shape (n_samples, n_features), the input data

        Returns:
            ndarray of (n_samples, n_components), the data after dimensionality reduction

        """
        n, d = X.shape
        assert d==self.d,\
            f"Input data should have a shape (n_samples, n_features) and n_features should be {self.d}"

        Xstar = X - np.outer(np.ones(n), self.mu)

        U = ridge_regression(
            self.components_.T, Xstar.T, self.ridge_alpha, solver="cholesky",
        )

        return U

    def fit_transform(self, dat):
        """
        Fit the model with X and apply the dimensionality reduction on X.

        Args:
            dat: ndarray of shape (n_samples, n_features)

        Returns:
            ndarray of (n_samples, n_components)

        """
        self.fit(dat)
        return self.transform(dat)

    def fine_tune_lambdas(self, dat, lambdas=np.arange(0, 0.00061, 0.0006 / 10)):
        # fine tune the Lambda values based on BICs following the paper
        BICs, zeros = [], []
        for lbd in lambdas:
            # print(f"Lambda: {lbd:.6f}")
            this_res = sparse_logistic_pca(
            dat, lbd=lbd, k=self.n_components, verbose=False,
            max_iters=self.max_iters, crit=self.crit,
            randstart=self.randstart, procrustes=self.procrustes,
            lasso=self.lasso,)
            BICs.append(this_res['BIC'])
            zeros.append(this_res['zeros'])
        best_ldb = lambdas[np.argmin(BICs)]
        return best_ldb, BICs, zeros

    def set_lambda(self,new_lbd):
        print(f"Setting lambda to: {new_lbd}")
        self.lbd = new_lbd

    def set_ridge_alpha(self, ridge_alpha):
        self.ridge_alpha = ridge_alpha

    def set_n_components(self, n_components):
        self.n_components = n_components

    def get_components(self):
        return self.components_

In [2]:
dat = np.random.randint(2, size=(40,16,),).astype(float)  

In [3]:
dat

array([[0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1.],
       [0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1.],
       [1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1.],
       [1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0.],
       [1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0.],
       [0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1.],
       [0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0.],
       [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1.],
       [0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0.],
       [0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1.],
       [1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1.],
       [0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1.,

In [4]:
# the binary matrix to fit, shape: (n_samples, n_features)
dat = np.random.randint(2, size=(40,16,),).astype(float)  

# create model and fit data
# lbd: lambda controlling sparsity of components
SLPCA = SparseLogisticPCA(n_components=2, lbd=0.0001)  
SLPCA.fit(dat, verbose=True)  

# the binary matrix to transform, shape: (n_samples, n_features)
X = np.random.randint(2, size=(20,16,),).astype(float)  

# get the transformed data
X_t = SLPCA.transform(X)

Iter: 0 - Loss: 0.5289, NegLogLike: 337.9820, Penalty: 0.5360 
Iter: 1 - Loss: 0.5213, NegLogLike: 333.0097, Penalty: 0.6203 
Iter: 2 - Loss: 0.5177, NegLogLike: 330.6690, Penalty: 0.6670 
Iter: 3 - Loss: 0.5153, NegLogLike: 329.0852, Penalty: 0.6990 
Iter: 4 - Loss: 0.5134, NegLogLike: 327.8378, Penalty: 0.7238 
Iter: 5 - Loss: 0.5118, NegLogLike: 326.7885, Penalty: 0.7447 
Iter: 6 - Loss: 0.5104, NegLogLike: 325.8790, Penalty: 0.7634 
Iter: 7 - Loss: 0.5092, NegLogLike: 325.0789, Penalty: 0.7812 
Iter: 8 - Loss: 0.5081, NegLogLike: 324.3695, Penalty: 0.7980 
Iter: 9 - Loss: 0.5071, NegLogLike: 323.7361, Penalty: 0.8140 
Iter: 10 - Loss: 0.5062, NegLogLike: 323.1679, Penalty: 0.8293 
Iter: 11 - Loss: 0.5055, NegLogLike: 322.6555, Penalty: 0.8440 
Iter: 12 - Loss: 0.5048, NegLogLike: 322.1912, Penalty: 0.8581 
Iter: 13 - Loss: 0.5041, NegLogLike: 321.7682, Penalty: 0.8718 
Iter: 14 - Loss: 0.5035, NegLogLike: 321.3807, Penalty: 0.8851 
Iter: 15 - Loss: 0.5030, NegLogLike: 321.0239, Pen

In [5]:
X_t

array([[-0.79515761, -0.75928224],
       [-0.8479762 , -1.08003798],
       [-0.60091912, -1.24437601],
       [-1.21029583,  0.13036661],
       [-0.71705681, -1.05627988],
       [-0.26945753, -0.62709579],
       [ 0.0625411 ,  0.14218722],
       [-1.5196023 , -0.35777352],
       [-0.39867368, -0.85970238],
       [-1.05767814, -0.15210979],
       [ 0.24221953, -0.4310016 ],
       [-0.18951689,  0.08387088],
       [-1.40656054, -0.11462958],
       [-0.31584435, -0.93718864],
       [ 0.05886169, -0.51000958],
       [-0.35764222, -0.22365524],
       [-1.12846303,  0.49009269],
       [-0.982654  , -0.18781202],
       [-0.21662577, -0.07776975],
       [-0.87780215,  0.00168608]])

In [6]:
components = SLPCA.get_components()
components

array([[-0.23191565, -0.02982951,  0.06358042, -0.10853536, -0.05287583,
         0.71656663,  0.41307586,  0.02726139,  0.1629159 , -0.43555671,
         0.09530327,  0.05897715,  0.05660401, -0.07334414,  0.0409295 ,
        -0.05285486],
       [-0.29585076,  0.05299171,  0.08320947, -0.0478146 , -0.01170234,
         0.51627725, -0.58702832, -0.20643897, -0.11959264,  0.23506211,
         0.02503766, -0.00091993,  0.02076631,  0.41781461,  0.03276973,
        -0.01724099]])

In [1]:
import anndata as ad 
cdata = ad.read('D://Test/scMeth/enhancer_c1_CG_paper.h5ad')

In [5]:
cdata.X.shape

(3379, 55017)

In [ ]:
# create model and fit data
# lbd: lambda controlling sparsity of components
SLPCA = SparseLogisticPCA(n_components=2, lbd=0.0001)  
SLPCA.fit(cdata.X, verbose=True)  

# the binary matrix to transform, shape: (n_samples, n_features)
X = np.random.randint(2, size=(20,16,),).astype(float)  

# get the transformed data
X_t = SLPCA.transform(X)

In [7]:
from sklearn.cluster import MiniBatchKMeans
from sklearn import decomposition

In [8]:
estimator = decomposition.MiniBatchDictionaryLearning(n_components=15, alpha=0.1,
                                                  n_iter=50, batch_size=3)

In [13]:
import numpy as np
from sklearn.decomposition import SparsePCA

In [14]:
# 创建Sparse PCA模型
sparse_pca = SparsePCA(n_components=10, alpha=0.1)  # 你可以根据需要选择合适的超参数

# 拟合Sparse PCA模型
sparse_pca.fit(cdata.X)

# 获取稀疏主成分和它们的权重
sparse_components = sparse_pca.components_

# 降维
X_transformed = sparse_pca.transform(cdata.X)

ValueError: Input X contains NaN.
SparsePCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [16]:
# import
from scipy.linalg import toeplitz
import numpy as np
from copy import copy

# simulate data
K = 10 - toeplitz(np.arange(10))
data1 = np.cumsum(np.random.multivariate_normal(np.zeros(10), K, 250), axis=0)
data1

array([[  1.849964  ,  -0.17153347,   0.8072775 , ...,  -1.89278527,
         -1.83773287,  -2.6370121 ],
       [  0.95847795,  -1.02198946,   0.7036406 , ...,   0.35725168,
          0.48607854,  -2.22351473],
       [ -2.70793834,  -4.00148546,  -1.72552943, ...,  -5.21196656,
         -6.34598292,  -9.52472873],
       ...,
       [-52.45279073, -40.56265838, -27.9396152 , ...,  -4.54400283,
        -22.98919745, -11.89009458],
       [-57.42504199, -46.91738255, -33.60712335, ...,  -8.83782935,
        -29.85530106, -15.58977452],
       [-59.68001976, -47.61288098, -33.3117344 , ..., -10.87402717,
        -30.72842098, -15.23069623]])

In [17]:
!pip install hypertools

     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     -------------------- ------------------- 30.7/59.7 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 59.7/59.7 kB 797.7 kB/s eta 0:00:00


In [19]:
# import
from scipy.linalg import toeplitz
import numpy as np
from copy import copy
import hypertools as hyp

# simulate data
K = 10 - toeplitz(np.arange(10))
data1 = np.cumsum(np.random.multivariate_normal(np.zeros(10), K, 250), axis=0)
data2 = copy(data1)

# simulate missing data
missing = .1
inds = [(i,j) for i in range(data2.shape[0]) for j in range(data2.shape[1])]
missing_data = [inds[i] for i in np.random.choice(int(len(inds)), int(len(inds)*missing))]
for i,j in missing_data:
    data2[i,j]=np.nan

# plot
hyp.plot([data1, data2], linestyle=['-',':'], legend=['Original', 'PPCA'])

D:\conda\envs\py39\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
D:\conda\envs\py39\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
D:\conda\envs\py39\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argument was no

In [20]:
!pip install ppca

In [21]:
from ppca import PPCA
ppca = PPCA()

In [23]:
abs = ppca.fit(data=cdata.X, d=100, verbose=True)

1.0
0.6622113523210857
1.7173829596225307
1.6082670019509013
0.40925052084155555
0.19915006723391904
0.11838535166400588
0.07813037029146175
0.05510118461279556
0.04073232683772576
0.03121244714005189
0.024618566245064644
0.01988980089605752
0.01640089306062298
0.013764276285280097
0.011729989022336795
0.010131530100216501
0.008854952377665981
0.007820442002301586
0.006970944534084644
0.00626493093693381
0.005671678416352188
0.005168122885478077
0.004736716836989174
0.004363946255991014
0.004039286555737487
0.0037544554503028404
0.003502873242171134
0.0032792656031614964
0.0030793692526183136
0.0028997112182822526
0.0027374411775569207
0.0025902033648075307
0.002456037881657158
0.0023333039447548387
0.002220619413233038
0.0021168133652631482
0.0020208884039081276
0.0019319902251453414
0.0018493835795976299
0.0017724326235384336
0.0017005846012760895
0.0016333572339986002
0.00157032755732045
0.0015111234580702515
0.0014554159314343895
0.001402913226370206
0.0013533556651301737
0.0013065

In [24]:
abs